In [ ]:
"""
What we want to achieve in this notebook:
- Pairwise Wilcoxon signed-rank + Holm correction (SciPy) + Critical Difference plot
"""
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

root_folder = os.getcwd()

DATA_PATH = "performance_comparison_gauss_1_clean.csv"      # path to your CSV
CSV_SEPARATOR = ","           # usually "," or ";"
DROP_AVERAGE_ROW = True       # drop last row if it is an average
LOWER_IS_BETTER = True       # True for errors (RMSE, MAE), False for scores
ALPHA = 0.05                  # significance level for our tests

df_path = os.path.join(root_folder, "data", "synthetic-golden-standard", "output-analysis")
data = pd.read_csv(os.path.join(df_path, DATA_PATH), sep=CSV_SEPARATOR)
print(data.head(2))
df = data[["HOG", "SCHARR", "SOBEL", "FIJI"]]
df

print(df.mean(axis=0))

### Friedman test + Nemenyi (post-hoc) test
After Friedman test rejects the hypothesis that all methods are equivalent, we can run the Nemenyi (post-hoc) test

In [ ]:
from scipy.stats import friedmanchisquare

# Perform Friedman test
# H0: All methods are on average the same
# We pass each column as a separate argument to friedmanchisquare
test_statistic, p_value = friedmanchisquare(*[df[col].values for col in df.columns])

print("=" * 60)
print("FRIEDMAN TEST")
print("=" * 60)
print(f"H0: All methods are sampled from the same population")
print(f"Test statistic: {test_statistic:.6f}")
print(f"P-value: {p_value:.6e}")
print(f"Significance level (α): {ALPHA}")
print()
if p_value < ALPHA:
    print(f"Result: REJECT H0 (p < {ALPHA})")
    print("Conclusion: There are significant differences between methods.")
else:
    print(f"Result: FAIL TO REJECT H0 (p ≥ {ALPHA})")
    print("Conclusion: No significant differences found between methods.")
print("=" * 60)
print()

Well, the rejection is pretty strong, so we can confidently move on with the post-hoc analaysis

### Nemenyi test


In [ ]:
from scipy.stats import rankdata, studentized_range

def mean_ranks(df: pd.DataFrame, higher_is_better: bool) -> pd.Series:
    """Mean ranks across rows; rank 1 = best."""
    x = df.to_numpy(dtype=float)
    ranks = np.zeros_like(x)
    for i in range(x.shape[0]):
        ranks[i] = rankdata(-x[i] if higher_is_better else x[i], method="average")
    return pd.Series(ranks.mean(axis=0), index=df.columns, name="mean_rank")


def nemenyi_cd(n_datasets: int, n_methods: int, alpha: float = 0.05) -> float:
    """
    Nemenyi critical difference for average ranks (Demšar 2006).
    CD = q_alpha * sqrt(k(k+1)/(6N))
    where q_alpha comes from the Studentized range distribution.
    """
    if n_datasets <= 1:
        raise ValueError("Need at least 2 datasets/rows to compute CD.")
    if n_methods <= 1:
        raise ValueError("Need at least 2 methods/columns to compute CD.")

    # Studentized range critical value q_alpha for k groups, infinite dof
    q_alpha = studentized_range.ppf(1 - alpha, n_methods, np.inf)
    se = np.sqrt(n_methods * (n_methods + 1) / (6.0 * n_datasets))
    return float(q_alpha * se)


def nemenyi_pvals(mean_rank: pd.Series, n_datasets: int) -> pd.DataFrame:
    """
    Pairwise p-values for Nemenyi test based on average ranks.
    p = 1 - CDF_studentized_range(q), where
    q = |ri - rj| / sqrt(k(k+1)/(6N))
    """
    methods = list(mean_rank.index)
    k = len(methods)
    se = np.sqrt(k * (k + 1) / (6.0 * n_datasets))

    pmat = np.ones((k, k), dtype=float)
    for i in range(k):
        for j in range(i + 1, k):
            q = abs(mean_rank.iloc[i] - mean_rank.iloc[j]) / se
            p = 1.0 - studentized_range.cdf(q, k, np.inf)
            pmat[i, j] = p
            pmat[j, i] = p

    return pd.DataFrame(pmat, index=methods, columns=methods)


def _contiguous_nonsig_segments(
    mean_rank_sorted: pd.Series, cd: float
) -> list[tuple[int, int]]:
    """
    Find contiguous segments (in sorted-by-rank order) where *all* pairs
    within the segment have rank differences <= CD (i.e., not significant
    under Nemenyi at alpha).

    Returns list of (start_idx, end_idx) inclusive indices in the sorted list.
    """
    r = mean_rank_sorted.to_numpy()
    k = len(r)
    segments: list[tuple[int, int]] = []

    # Brute-force maximal contiguous segments:
    # segment [i, j] is valid if max(r[i..j]) - min(r[i..j]) <= CD,
    # but since r is sorted increasing, that's r[j]-r[i] <= CD.
    i = 0
    while i < k:
        j = i
        while j + 1 < k and (r[j + 1] - r[i]) <= cd:
            j += 1
        # Only draw segments of length >= 2
        if j > i:
            segments.append((i, j))
        i += 1

    # Reduce overlaps a bit: keep only segments that are not strict subsegments
    reduced: list[tuple[int, int]] = []
    for a, b in segments:
        is_sub = any((c <= a and b <= d) and ((c, d) != (a, b)) for c, d in segments)
        if not is_sub:
            reduced.append((a, b))

    # Sort by length descending then by start
    reduced.sort(key=lambda t: (-(t[1] - t[0]), t[0]))
    return reduced


def critical_difference_diagram(
    df: pd.DataFrame,
    *,
    higher_is_better: bool = True,
    alpha: float = 0.05,
    title: str | None = None,
    figsize: tuple[float, float] = (10, 3.2),
    label_rotation: float = 30.0,
) -> tuple[plt.Figure, plt.Axes]:
    """
    Create a Demšar-style Critical Difference (CD) diagram with Nemenyi post-hoc.
    df: rows = datasets, cols = methods.
    """
    if df.isna().any().any():
        raise ValueError("df contains NaNs. Please impute/drop before plotting.")

    n_datasets = df.shape[0]
    n_methods = df.shape[1]

    mr = mean_ranks(df, higher_is_better=higher_is_better)
    mr_sorted = mr.sort_values()  # rank 1 best, so smaller is better
    cd = nemenyi_cd(n_datasets=n_datasets, n_methods=n_methods, alpha=alpha)

    # Prepare plot
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_axis_off()

    # Axis range (ranks are between 1..k)
    k = n_methods
    xmin, xmax = 1.0, float(k)
    pad = 0.6
    ax.set_xlim(xmin - pad, xmax + pad)
    ax.set_ylim(0, 1)

    # Baseline y levels
    y_axis = 0.35
    y_labels = 0.18
    y_groups_top = 0.62

    # Draw rank axis
    ax.hlines(y_axis, xmin, xmax, linewidth=1.2)
    for r in range(1, k + 1):
        ax.vlines(r, y_axis - 0.02, y_axis + 0.02, linewidth=1.0)
        ax.text(r, y_axis - 0.07, str(r), ha="center", va="top", fontsize=12)


    # if ranks are too close, spread them a bit for better visibility. Push them right
    # the later rank further down ( and so on, domino effect if needed)
    plot_ranks = list(mr_sorted)

    prev_rank = 0.5 # min rank is 1, so this never triggers in the beginning
    for i, r in enumerate(plot_ranks):
        if r - prev_rank < 0.25:
            r += 0.25 - (r - prev_rank)
            plot_ranks[i] = r
        prev_rank = r
    print(plot_ranks)
    # Method markers + labels
    i = 0
    for name, rank in mr_sorted.items():
        ax.plot([plot_ranks[i]], [y_axis], marker="o", markersize=12)
        ax.text(
            plot_ranks[i],
            y_labels,
            name,
            ha="right",
            va="center",
            rotation=label_rotation,
            fontsize=14,
        )
        i += 1

    # CD bar (like Orange)
    cd_x0 = xmin
    cd_x1 = min(xmax, xmin + cd)
    ax.hlines(0.84, cd_x0, cd_x1, linewidth=2.0)
    ax.vlines(cd_x0, 0.82, 0.86, linewidth=2.0)
    ax.vlines(cd_x1, 0.82, 0.86, linewidth=2.0)
    ax.text((cd_x0 + cd_x1) / 2, 0.88, f"CD = {cd:.3f}", ha="center", va="bottom", fontsize=14)
    # if needed, we can add α={alpha} level as well
    # Non-significant groups as thick bars above axis
    segments = _contiguous_nonsig_segments(mr_sorted, cd=cd)
    # stack bars to avoid overlap
    level_step = 0.06
    used_levels: list[list[tuple[float, float]]] = []

    def place_segment(x0: float, x1: float) -> float:
        for level_idx, intervals in enumerate(used_levels):
            if all(x1 < a or x0 > b for a, b in intervals):
                intervals.append((x0, x1))
                return y_groups_top + level_idx * level_step
        used_levels.append([(x0, x1)])
        return y_groups_top + (len(used_levels) - 1) * level_step

    for i, j in segments:
        ranks = mr_sorted.to_numpy()
        x0, x1 = float(ranks[i]), float(ranks[j])
        y = place_segment(x0, x1)
        ax.hlines(y, x0, x1, linewidth=4.0)

    if title is None:
        title = "Critical Difference Diagram (Nemenyi)"
    ax.text((xmin + xmax) / 2, 1.1, title, ha="center", va="top", fontsize=16)

    return fig, ax

Visualize results with critical difference plots:

In [ ]:
fig, ax = critical_difference_diagram(df, higher_is_better=not LOWER_IS_BETTER, alpha=ALPHA)
plt.tight_layout()
plt.show()

In [ ]:
# Compute pairwise p-values for Nemenyi test
n_datasets = df.shape[0]
mr = mean_ranks(df, higher_is_better=not LOWER_IS_BETTER)
pmat_nemenyi = nemenyi_pvals(mr, n_datasets)

print("=" * 80)
print("NEMENYI POST-HOC TEST - PAIRWISE P-VALUES")
print("=" * 80)
print(f"Significance level (α): {ALPHA}")
print(f"Critical difference (CD): {nemenyi_cd(n_datasets, df.shape[1], ALPHA):.3f}")
print()
print("P-values for all pairwise comparisons:")
print("(P > 0.05: methods are connected in the CD diagram)")
print("(P ≤ 0.05: methods are NOT connected in the CD diagram)")
print("-" * 80)

methods = list(pmat_nemenyi.index)
for i in range(len(methods)):
    for j in range(i + 1, len(methods)):
        p_val = pmat_nemenyi.iloc[i, j]
        print(f"{methods[i]:25s} vs {methods[j]:25s} : p = {p_val:8f}")

print("=" * 80)
print()
print("Full pairwise p-value matrix:")
print(pmat_nemenyi.round(6))

## Wilcoxon Signed-Rank Test with Holm Correction

Pairwise Wilcoxon signed-rank test for all method comparisons with Holm-Bonferroni correction.

In [ ]:
from scipy.stats import wilcoxon
from itertools import combinations

def wilcoxon_pairwise_with_holm(df: pd.DataFrame, alpha: float = 0.05) -> tuple[pd.DataFrame, list, list]:
    """
    Perform pairwise Wilcoxon signed-rank tests with Holm-Bonferroni correction.

    Parameters:
    -----------
    df : pd.DataFrame
        Rows = datasets/samples, Columns = methods/algorithms
    alpha : float
        Significance level

    Returns:
    --------
    pmat : pd.DataFrame
        Matrix of p-values (uncorrected)
    summary : list
        List of dictionaries with results
    results : list
        Detailed results with corrections
    """
    methods = df.columns.tolist()
    k = len(methods)

    # Generate all pairwise comparisons
    pairs = list(combinations(range(k), 2))
    n_comparisons = len(pairs)

    # Collect results
    results = []
    p_values = []

    for i, j in pairs:
        method_i = methods[i]
        method_j = methods[j]

        # Wilcoxon signed-rank test: H0 that paired samples have same distribution
        stat, p_val = wilcoxon(df.iloc[:, i].values, df.iloc[:, j].values, alternative='two-sided')
        p_values.append(p_val)
        results.append({
            'method1': method_i,
            'method2': method_j,
            'statistic': stat,
            'p_value': p_val,
            'pair_idx': (i, j)
        })

    # Holm-Bonferroni correction
    # Sort p-values in ascending order while keeping track of original indices
    sorted_indices = np.argsort(p_values)
    sorted_p_values = np.array(p_values)[sorted_indices]

    # Apply Holm correction: multiply each p-value by (n_comparisons - rank + 1)
    for rank, p_idx in enumerate(sorted_indices):
        corrected_p = min(1.0, sorted_p_values[rank] * (n_comparisons - rank))
        results[p_idx]['p_value_corrected'] = corrected_p
        results[p_idx]['rejected'] = corrected_p < alpha

    # Create p-value matrix
    pmat = np.ones((k, k), dtype=float)
    for i, j in pairs:
        p_val = results[pairs.index((i, j))]['p_value']
        pmat[i, j] = p_val
        pmat[j, i] = p_val

    pmat_df = pd.DataFrame(pmat, index=methods, columns=methods)

    # Create summary with corrections
    summary = []
    for res in results:
        summary.append({
            'Method 1': res['method1'],
            'Method 2': res['method2'],
            'p-value': f"{res['p_value']:.6f}",
            'p-value (Holm)': f"{res['p_value_corrected']:.6f}",
            'Significant': res['rejected']
        })

    return pmat_df, summary, results


# Run Wilcoxon test
pmat_wilcoxon, summary_wilcoxon, results_wilcoxon = wilcoxon_pairwise_with_holm(df, alpha=ALPHA)

print("=" * 80)
print("WILCOXON SIGNED-RANK TEST (Pairwise) WITH HOLM-BONFERRONI CORRECTION")
print("=" * 80)
print(pd.DataFrame(summary_wilcoxon))
print("\n" + "=" * 80)
print("Interpretation: Significant=True means the two methods differ significantly")
print("=" * 80)

In [ ]:
def wilcoxon_cd_diagram(
    df: pd.DataFrame,
    alpha: float = 0.05,
    title: str | None = None,
    figsize: tuple[float, float] = (10, 3.2),
    label_rotation: float = 30.0,
) -> tuple[plt.Figure, plt.Axes]:
    """
    Create a Critical Difference diagram for Wilcoxon signed-rank test results.
    Methods are ordered by median performance, with non-significant groups connected.

    Parameters:
    -----------
    df : pd.DataFrame
        Rows = datasets/samples, Columns = methods/algorithms
    alpha : float
        Significance level for Holm-corrected Wilcoxon test
    title : str, optional
        Plot title
    figsize : tuple, optional
        Figure size
    label_rotation : float, optional
        Rotation angle for method labels

    Returns:
    --------
    fig, ax : matplotlib figure and axes
    """
    if df.isna().any().any():
        raise ValueError("df contains NaNs. Please impute/drop before plotting.")

    # Calculate median performance for each method (for ranking/ordering)
    medians = df.median()
    median_sorted = medians.sort_values()

    # Run Wilcoxon test
    pmat_df, summary, results = wilcoxon_pairwise_with_holm(df, alpha=alpha)

    # Identify non-significant pairs (methods that are not significantly different)
    methods = df.columns.tolist()
    k = len(methods)

    # Build graph of non-significant connections
    not_sig_pairs = []
    for res in results:
        if not res['rejected']:  # p_value_corrected >= alpha
            i = methods.index(res['method1'])
            j = methods.index(res['method2'])
            not_sig_pairs.append((i, j))

    # Find connected components (groups of non-significantly different methods)
    # Using simple union-find approach
    parent = list(range(k))

    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[px] = py

    for i, j in not_sig_pairs:
        union(i, j)

    # Group methods by their connected component
    groups = {}
    for i in range(k):
        root = find(i)
        if root not in groups:
            groups[root] = []
        groups[root].append(methods[i])

    # Prepare plot
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_axis_off()

    # Axis range
    xmin, xmax = 1.0, float(k)
    pad = 0.6
    ax.set_xlim(xmin - pad, xmax + pad)
    ax.set_ylim(0, 1)

    # Baseline y levels
    y_axis = 0.35
    y_labels = 0.18
    y_groups_top = 0.62

    # Draw rank axis with method names ordered by median
    ax.hlines(y_axis, xmin, xmax, linewidth=1.2)

    for idx, (name, rank) in enumerate(median_sorted.items()):
        x_pos = idx + 1
        ax.vlines(x_pos, y_axis - 0.02, y_axis + 0.02, linewidth=1.0)
        ax.text(x_pos, y_axis - 0.07, str(idx + 1), ha="center", va="top", fontsize=12)

    # Method markers + labels
    for idx, (name, rank) in enumerate(median_sorted.items()):
        x_pos = idx + 1
        ax.plot([x_pos], [y_axis], marker="o", markersize=12, color='C0')
        ax.text(
            x_pos,
            y_labels,
            name,
            ha="right",
            va="center",
            rotation=label_rotation,
            fontsize=14,
        )

    # Draw non-significant groups as horizontal lines
    # Map method names to x positions
    name_to_xpos = {}
    for idx, (name, _) in enumerate(median_sorted.items()):
        name_to_xpos[name] = idx + 1

    # Draw connecting lines for non-significant groups
    level_step = 0.06
    used_levels = []

    def place_segment(x0: float, x1: float) -> float:
        for level_idx, intervals in enumerate(used_levels):
            if all(x1 < a - 0.05 or x0 > b + 0.05 for a, b in intervals):
                intervals.append((x0, x1))
                return y_groups_top + level_idx * level_step
        used_levels.append([(x0, x1)])
        return y_groups_top + (len(used_levels) - 1) * level_step

    # For each group, draw a line connecting all members
    for group_methods in groups.values():
        if len(group_methods) > 1:
            x_positions = [name_to_xpos[m] for m in group_methods]
            x_min = min(x_positions)
            x_max = max(x_positions)
            y = place_segment(x_min, x_max)
            ax.hlines(y, x_min, x_max, linewidth=4.0, color='C0')

    if title is None:
        title = f"Critical Difference Diagram (Wilcoxon signed-rank, α={alpha}, Holm corrected)"
    ax.text((xmin + xmax) / 2, 1.1, title, ha="center", va="top", fontsize=16)

    return fig, ax


print("Wilcoxon CD diagram function defined and ready.")

In [ ]:
fig, ax = wilcoxon_cd_diagram(df, alpha=ALPHA)
plt.tight_layout()
plt.show()

Everything is significant according to this test (hence, the lame plot)
Plus, the average ranking is not relevant for this test, as differences within measurements are used instead.